In [1]:
"""
Простая CNN модель для классификации изображений.
Архитектура: 3 сверточных блока + классификатор.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F


class SimpleCNN(nn.Module):
    """
    Простая сверточная нейронная сеть для классификации изображений.

    Архитектура:
    - 3 сверточных блока (Conv2d + BatchNorm + ReLU + MaxPool)
    - Adaptive Average Pooling для гибкости к размеру входа
    - Fully Connected классификатор с Dropout

    Args:
        num_classes: Количество классов для классификации
        dropout: Вероятность dropout (по умолчанию 0.5)
    """

    def __init__(self, num_classes: int, dropout: float = 0.5):
        super(SimpleCNN, self).__init__()

        # Первый сверточный блок: 3 -> 64 каналов
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Второй сверточный блок: 64 -> 128 каналов
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Третий сверточный блок: 128 -> 256 каналов
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Adaptive pooling для получения фиксированного размера
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))

        # Fully connected слои
        self.fc1 = nn.Linear(256 * 7 * 7, 512)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(512, num_classes)

        # Инициализация весов
        self._initialize_weights()

    def _initialize_weights(self):
        """Инициализация весов модели (Kaiming initialization для ReLU)."""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Прямой проход через сеть.

        Args:
            x: Входной тензор размера (batch_size, 3, H, W)

        Returns:
            Выходной тензор размера (batch_size, num_classes)
        """
        # Первый блок
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool1(x)

        # Второй блок
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool2(x)

        # Третий блок
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool3(x)

        # Adaptive pooling
        x = self.adaptive_pool(x)

        # Flatten
        x = x.view(x.size(0), -1)

        # Fully connected слои
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x

    def get_num_parameters(self) -> int:
        """Возвращает количество обучаемых параметров."""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)


In [2]:
"""
ViT-Tiny модель с linear probe для классификации изображений.
Использует предобученный backbone с замороженными весами.
"""

import torch
import torch.nn as nn
import timm
from typing import Optional


class ViTLinearProbe(nn.Module):
    """
    Vision Transformer Tiny с linear probe.

    Загружает предобученную ViT-Tiny модель, замораживает все слои backbone
    и обучает только линейную голову (classifier).

    Args:
        num_classes: Количество классов для классификации
        pretrained: Использовать ли предобученные веса (по умолчанию True)
        model_name: Название модели из timm (по умолчанию 'vit_tiny_patch16_224')
    """

    def __init__(
        self,
        num_classes: int,
        pretrained: bool = True,
        model_name: str = 'vit_tiny_patch16_224'
    ):
        super(ViTLinearProbe, self).__init__()

        # Загружаем предобученную ViT-Tiny модель
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=0  # Убираем классификатор
        )

        # Замораживаем все параметры backbone
        for param in self.backbone.parameters():
            param.requires_grad = False

        # Получаем размерность выходных признаков
        self.feature_dim = self.backbone.num_features

        # Создаем новую линейную голову
        self.classifier = nn.Linear(self.feature_dim, num_classes)

        # Инициализация весов классификатора
        nn.init.normal_(self.classifier.weight, 0, 0.01)
        nn.init.constant_(self.classifier.bias, 0)

        print(f"ViT-Tiny модель загружена:")
        print(f"  - Модель: {model_name}")
        print(f"  - Предобучена: {pretrained}")
        print(f"  - Размерность признаков: {self.feature_dim}")
        print(f"  - Количество классов: {num_classes}")
        print(f"  - Backbone заморожен: все параметры requires_grad=False")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Прямой проход через сеть.

        Args:
            x: Входной тензор размера (batch_size, 3, 224, 224)

        Returns:
            Выходной тензор размера (batch_size, num_classes)
        """
        # Извлекаем признаки через backbone (без градиентов)
        with torch.no_grad():
            features = self.backbone(x)

        # Классификация через линейную голову
        output = self.classifier(features)

        return output

    def unfreeze_backbone(self):
        """Размораживает backbone для fine-tuning (опционально)."""
        for param in self.backbone.parameters():
            param.requires_grad = True
        print("Backbone разморожен для fine-tuning")

    def get_num_parameters(self, trainable_only: bool = True) -> int:
        """
        Возвращает количество параметров.

        Args:
            trainable_only: Если True, считает только обучаемые параметры

        Returns:
            Количество параметров
        """
        if trainable_only:
            return sum(p.numel() for p in self.parameters() if p.requires_grad)
        else:
            return sum(p.numel() for p in self.parameters())

    def get_parameter_stats(self) -> dict:
        """Возвращает статистику по параметрам модели."""
        total_params = self.get_num_parameters(trainable_only=False)
        trainable_params = self.get_num_parameters(trainable_only=True)
        frozen_params = total_params - trainable_params

        return {
            'total': total_params,
            'trainable': trainable_params,
            'frozen': frozen_params,
            'trainable_percentage': (trainable_params / total_params) * 100
        }


class ViTLinearProbeWithDropout(ViTLinearProbe):
    """
    ViT-Tiny с linear probe и dropout для регуляризации.

    Args:
        num_classes: Количество классов для классификации
        pretrained: Использовать ли предобученные веса
        model_name: Название модели из timm
        dropout: Вероятность dropout перед классификатором
    """

    def __init__(
        self,
        num_classes: int,
        pretrained: bool = True,
        model_name: str = 'vit_tiny_patch16_224',
        dropout: float = 0.1
    ):
        super().__init__(num_classes, pretrained, model_name)

        # Заменяем простой классификатор на классификатор с dropout
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(self.feature_dim, num_classes)
        )

        # Инициализация весов
        for m in self.classifier.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

        print(f"  - Dropout: {dropout}")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Прямой проход с dropout."""
        # Извлекаем признаки через backbone (без градиентов)
        with torch.no_grad():
            features = self.backbone(x)

        # Классификация через голову с dropout
        output = self.classifier(features)

        return output


In [3]:
from torchvision.datasets import STL10
from pathlib import Path
import numpy as np

def export_stl10_to_imagefolder(
    root: Path,
    train_root: Path,
    val_root: Path,
    val_ratio: float = 0.2,
    max_per_class: Optional[int] = None,
    seed: int = 42
) -> None:
    """
    Export STL10 dataset to ImageFolder structure with train/val splits.

    Args:
        root: Root directory for the dataset
        val_ratio: Proportion of data to use for validation
        max_per_class: Maximum number of samples per class (None for all)
        seed: Random seed for reproducibility
    """

    # Load dataset
    ds = STL10(str(root), split='train', download=True)
    classes = list(ds.classes)

    # Create directory structure
    _create_class_directories(train_root, val_root, classes)

    # Process each class
    labels = np.array(ds.labels)

    for cls_idx, cls_name in enumerate(classes):
        # Get indices for current class
        cls_mask = labels == cls_idx
        cls_indices = np.where(cls_mask)[0]

        # Shuffle indices
        rng = np.random.default_rng(seed + cls_idx)
        rng.shuffle(cls_indices)

        # Limit samples if specified
        if max_per_class is not None:
            cls_indices = cls_indices[:max_per_class]

        # Split into train and validation
        n_val = int(round(len(cls_indices) * val_ratio))
        val_indices = cls_indices[:n_val]
        train_indices = cls_indices[n_val:]

        # Save images
        _save_images(ds, train_root / cls_name, train_indices)
        _save_images(ds, val_root / cls_name, val_indices)


def _create_class_directories(train_root: Path, val_root: Path, classes: list) -> None:
    """Create directory structure for all classes."""
    for cls_name in classes:
        (train_root / cls_name).mkdir(parents=True, exist_ok=True)
        (val_root / cls_name).mkdir(parents=True, exist_ok=True)


def _save_images(dataset, output_dir: Path, indices: np.ndarray) -> None:
    """Save images from dataset to output directory."""
    for idx in indices:
        output_path = output_dir / f'{idx:06d}.png'
        if not output_path.exists():
            img, _ = dataset[int(idx)]
            img.save(output_path)

In [4]:
"""
Модуль для подготовки данных и аугментаций.
Реализует загрузку датасета в формате ImageFolder с аугментациями.
"""

import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import random
import numpy as np
from typing import Tuple, Optional


def set_seed(seed: int = 42):
    """
    Устанавливает фиксированный seed для воспроизводимости результатов.

    Args:
        seed: Значение seed для генераторов случайных чисел
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def get_transforms(train: bool = True) -> transforms.Compose:
    """
    Возвращает композицию трансформаций для обучающей или валидационной выборки.

    Args:
        train: Если True, применяются аугментации для обучения

    Returns:
        Композиция трансформаций
    """
    # Нормализация ImageNet (стандартные значения для предобученных моделей)
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )

    if train:
        # Аугментации для обучающей выборки
        return transforms.Compose([
            transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(),
            normalize
        ])
    else:
        # Трансформации для валидационной выборки (без аугментаций)
        return transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize
        ])


def get_dataloaders(
    data_dir: str,
    batch_size: int = 32,
    num_workers: int = 4,
    seed: int = 42
) -> Tuple[DataLoader, DataLoader, int]:
    """
    Создает DataLoader'ы для обучающей и валидационной выборок.

    Args:
        data_dir: Путь к директории с данными (должна содержать train/ и val/)
        batch_size: Размер батча
        num_workers: Количество процессов для загрузки данных
        seed: Seed для воспроизводимости

    Returns:
        Кортеж (train_loader, val_loader, num_classes)
    """
    set_seed(seed)

    # Создаем датасеты с соответствующими трансформациями
    train_dataset = datasets.ImageFolder(
        root=f"{data_dir}/train",
        transform=get_transforms(train=True)
    )

    val_dataset = datasets.ImageFolder(
        root=f"{data_dir}/val",
        transform=get_transforms(train=False)
    )

    num_classes = len(train_dataset.classes)

    print(f"Загружен датасет:")
    print(f"  - Классы: {train_dataset.classes}")
    print(f"  - Количество классов: {num_classes}")
    print(f"  - Обучающая выборка: {len(train_dataset)} изображений")
    print(f"  - Валидационная выборка: {len(val_dataset)} изображений")

    # Создаем DataLoader'ы
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=True  # Для стабильности batch normalization
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    return train_loader, val_loader, num_classes


def get_sanity_check_loader(
    data_dir: str,
    num_batches: int = 5,
    batch_size: int = 32,
    seed: int = 42
) -> Tuple[DataLoader, int]:
    """
    Создает DataLoader для sanity check (overfit на нескольких батчах).

    Args:
        data_dir: Путь к директории с данными
        num_batches: Количество батчей для overfit
        batch_size: Размер батча
        seed: Seed для воспроизводимости

    Returns:
        Кортеж (sanity_loader, num_classes)
    """
    set_seed(seed)

    # Загружаем полный датасет
    train_dataset = datasets.ImageFolder(
        root=f"{data_dir}/train",
        transform=get_transforms(train=True)
    )

    num_classes = len(train_dataset.classes)

    # Берем только первые num_batches * batch_size примеров
    num_samples = num_batches * batch_size
    indices = list(range(min(num_samples, len(train_dataset))))

    subset = Subset(train_dataset, indices)

    print(f"Sanity check режим:")
    print(f"  - Количество примеров: {len(subset)}")
    print(f"  - Количество батчей: {num_batches}")
    print(f"  - Размер батча: {batch_size}")

    # Создаем DataLoader (shuffle=True для разнообразия)
    sanity_loader = DataLoader(
        subset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True,
        drop_last=True
    )

    return sanity_loader, num_classes


def verify_dataset(data_dir: str) -> bool:
    """
    Проверяет корректность структуры датасета.

    Args:
        data_dir: Путь к директории с данными

    Returns:
        True если датасет корректен, иначе False
    """
    import os

    train_dir = os.path.join(data_dir, 'train')
    val_dir = os.path.join(data_dir, 'val')

    if not os.path.exists(train_dir):
        print(f"Ошибка: директория {train_dir} не найдена")
        return False

    if not os.path.exists(val_dir):
        print(f"Ошибка: директория {val_dir} не найдена")
        return False

    # Проверяем количество классов
    train_classes = [d for d in os.listdir(train_dir)
                     if os.path.isdir(os.path.join(train_dir, d))]
    val_classes = [d for d in os.listdir(val_dir)
                   if os.path.isdir(os.path.join(val_dir, d))]

    if len(train_classes) < 5:
        print(f"Ошибка: недостаточно классов в train ({len(train_classes)} < 5)")
        return False

    if set(train_classes) != set(val_classes):
        print("Предупреждение: классы в train и val не совпадают")

    # Проверяем количество изображений в каждом классе
    for class_name in train_classes:
        class_dir = os.path.join(train_dir, class_name)
        num_images = len([f for f in os.listdir(class_dir)
                         if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        if num_images < 100:
            print(f"Предупреждение: класс {class_name} содержит только {num_images} изображений (< 100)")

    print("Датасет прошел базовую проверку")
    return True


from pathlib import Path
from torchvision.datasets import STL10, ImageFolder

DIR_PATH = Path("./data")
DIR_PATH.mkdir(exist_ok=True)

TRAIN_ROOT = DIR_PATH / 'train'
TRAIN_ROOT.mkdir(exist_ok=True)
VAL_ROOT = DIR_PATH / 'val'
VAL_ROOT.mkdir(exist_ok=True)

export_stl10_to_imagefolder(DIR_PATH, TRAIN_ROOT, VAL_ROOT)

# Проверяем датасет
if verify_dataset(DIR_PATH):
    # Создаем DataLoader'ы для проверки
    train_loader, val_loader, num_classes = get_dataloaders(
        str(DIR_PATH),
        batch_size=4,
        num_workers=2
    )

    # Проверяем первый батч
    images, labels = next(iter(train_loader))
    print(f"\nПример батча:")
    print(f"  - Размер изображений: {images.shape}")
    print(f"  - Размер меток: {labels.shape}")
    print(f"  - Метки в батче: {labels.tolist()}")


100%|██████████| 2.64G/2.64G [00:43<00:00, 61.2MB/s]


Датасет прошел базовую проверку
Загружен датасет:
  - Классы: ['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']
  - Количество классов: 10
  - Обучающая выборка: 4000 изображений
  - Валидационная выборка: 1000 изображений

Пример батча:
  - Размер изображений: torch.Size([4, 3, 224, 224])
  - Размер меток: torch.Size([4])
  - Метки в батче: [2, 9, 2, 0]


In [5]:
#import shutil
#folder_path = './data'
#shutil.rmtree(folder_path)

In [6]:
"""
Тренировочный цикл для CNN и ViT-Tiny моделей.
Включает sanity checks, логирование в TensorBoard, сохранение чекпоинтов.
"""

import os
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

def train_epoch(
    model: nn.Module,
    train_loader,
    criterion,
    optimizer,
    device: torch.device,
    epoch: int,
    writer: SummaryWriter,
    global_step: int
) -> tuple:
    """
    Обучение модели на одной эпохе.

    Args:
        model: Модель для обучения
        train_loader: DataLoader с обучающими данными
        criterion: Функция потерь
        optimizer: Оптимизатор
        device: Устройство (CPU/GPU)
        epoch: Номер текущей эпохи
        writer: TensorBoard writer
        global_step: Глобальный шаг для логирования

    Returns:
        Кортеж (средний loss, accuracy, новый global_step)
    """
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc=f'Epoch {epoch} [Train]')

    for batch_idx, (images, labels) in enumerate(pbar):
        images, labels = images.to(device), labels.to(device)

        # Прямой проход
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Обратный проход
        loss.backward()
        optimizer.step()

        # Статистика
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        # Логирование в TensorBoard
        if batch_idx % 10 == 0:
            writer.add_scalar('Train/Loss_step', loss.item(), global_step)
            writer.add_scalar('Train/Accuracy_step', 100. * correct / total, global_step)
            writer.add_scalar('Train/LR', optimizer.param_groups[0]['lr'], global_step)

        global_step += 1

        # Обновление progress bar
        pbar.set_postfix({
            'loss': f'{running_loss / (batch_idx + 1):.4f}',
            'acc': f'{100. * correct / total:.2f}%'
        })

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc, global_step


def validate(
    model: nn.Module,
    val_loader,
    criterion,
    device: torch.device,
    epoch: int,
    writer: SummaryWriter
) -> tuple:
    """
    Валидация модели.

    Args:
        model: Модель для валидации
        val_loader: DataLoader с валидационными данными
        criterion: Функция потерь
        device: Устройство (CPU/GPU)
        epoch: Номер текущей эпохи
        writer: TensorBoard writer

    Returns:
        Кортеж (средний loss, accuracy)
    """
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        pbar = tqdm(val_loader, desc=f'Epoch {epoch} [Val]')

        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            pbar.set_postfix({
                'loss': f'{running_loss / (pbar.n + 1):.4f}',
                'acc': f'{100. * correct / total:.2f}%'
            })

    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc


def log_model_weights_and_gradients(
    model: nn.Module,
    writer: SummaryWriter,
    epoch: int
):
    """
    Логирование гистограмм весов и градиентов в TensorBoard.

    Args:
        model: Модель
        writer: TensorBoard writer
        epoch: Номер эпохи
    """
    for name, param in model.named_parameters():
        if param.requires_grad:
            # Логируем веса
            writer.add_histogram(f'Weights/{name}', param.data, epoch)

            # Логируем градиенты (если есть)
            if param.grad is not None:
                writer.add_histogram(f'Gradients/{name}', param.grad, epoch)


def sanity_check(
    model: nn.Module,
    sanity_loader,
    criterion,
    optimizer,
    device: torch.device,
    num_epochs: int,
    writer: SummaryWriter
):
    """
    Sanity check: overfit на нескольких батчах.
    Модель должна достичь ~100% accuracy на этих данных.

    Args:
        model: Модель для проверки
        sanity_loader: DataLoader с небольшим количеством данных
        criterion: Функция потерь
        optimizer: Оптимизатор
        device: Устройство
        num_epochs: Количество эпох для overfit
        writer: TensorBoard writer
    """
    print("\n" + "="*60)
    print("SANITY CHECK: Overfit на нескольких батчах")
    print("="*60)

    model.train()
    global_step = 0

    for epoch in range(1, num_epochs + 1):
        running_loss = 0.0
        correct = 0
        total = 0

        for batch_idx, (images, labels) in enumerate(sanity_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            global_step += 1

        epoch_loss = running_loss / len(sanity_loader)
        epoch_acc = 100. * correct / total

        # Логирование
        writer.add_scalar('SanityCheck/Loss', epoch_loss, epoch)
        writer.add_scalar('SanityCheck/Accuracy', epoch_acc, epoch)

        if epoch % 10 == 0 or epoch == 1:
            print(f"Epoch {epoch}/{num_epochs} - Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.2f}%")

        # Если достигли 100% accuracy, sanity check пройден
        if epoch_acc >= 99.9:
            print(f"\n✓ Sanity check ПРОЙДЕН на эпохе {epoch}!")
            print(f"  Модель достигла {epoch_acc:.2f}% accuracy на обучающих данных")
            break
    else:
        print(f"\n✗ Sanity check НЕ ПРОЙДЕН!")
        print(f"  Финальная accuracy: {epoch_acc:.2f}% (ожидалось ~100%)")
        print(f"  Возможные проблемы: слишком маленький learning rate, ошибки в модели")


def train_model(
    model_name, data_dir, epochs, batch_size, lr, seed, num_workers, sanity_check_bool, num_batches
):
    """
    Основная функция обучения модели.

    Args:
        args: Аргументы командной строки
    """
    # Установка seed для воспроизводимости
    set_seed(seed)

    # Устройство
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Используется устройство: {device}")

    # Создание директорий
    os.makedirs('checkpoints', exist_ok=True)
    os.makedirs('runs', exist_ok=True)

    # TensorBoard writer
    run_name = f"{model_name}_{time.strftime('%Y%m%d_%H%M%S')}"
    if sanity_check_bool:
        run_name += "_sanity"
    writer = SummaryWriter(f'runs/{run_name}')

    # Загрузка данных
    if sanity_check_bool:
        print("\nРежим: SANITY CHECK")
        train_loader, num_classes = get_sanity_check_loader(
            data_dir,
            num_batches=num_batches,
            batch_size=batch_size,
            seed=seed
        )
        val_loader = None
    else:
        print("\nРежим: ОБЫЧНОЕ ОБУЧЕНИЕ")
        train_loader, val_loader, num_classes = get_dataloaders(
            data_dir,
            batch_size=batch_size,
            num_workers=num_workers,
            seed=seed
        )

    # Создание модели
    print(f"\nСоздание модели: {model_name}")
    if model_name == 'cnn':
        model = SimpleCNN(num_classes=num_classes, dropout=0.5)
    elif model_name == 'vit':
        model = ViTLinearProbe(num_classes=num_classes, pretrained=True)
    else:
        raise ValueError(f"Неизвестная модель: {model_name}")

    model = model.to(device)

    # Вывод информации о модели
    if hasattr(model, 'get_num_parameters'):
        num_params = model.get_num_parameters()
        print(f"Количество обучаемых параметров: {num_params:,}")

    if hasattr(model, 'get_parameter_stats'):
        stats = model.get_parameter_stats()
        print(f"Статистика параметров:")
        print(f"  - Всего: {stats['total']:,}")
        print(f"  - Обучаемых: {stats['trainable']:,}")
        print(f"  - Замороженных: {stats['frozen']:,}")

    # Функция потерь
    criterion = nn.CrossEntropyLoss()

    # Оптимизатор
    if model_name == 'cnn':
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    else:  # ViT
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)

    # Learning rate scheduler
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)

    # Sanity check
    if sanity_check_bool:
        sanity_check(
            model, train_loader, criterion, optimizer,
            device, epochs, writer
        )
        writer.close()
        return

    # Обучение
    print(f"\nНачало обучения на {epochs} эпох")
    print("="*60)

    best_val_acc = 0.0
    global_step = 0

    for epoch in range(1, epochs + 1):
        # Обучение
        train_loss, train_acc, global_step = train_epoch(
            model, train_loader, criterion, optimizer,
            device, epoch, writer, global_step
        )

        # Валидация
        val_loss, val_acc = validate(
            model, val_loader, criterion, device, epoch, writer
        )

        # Обновление learning rate
        scheduler.step()

        # Логирование эпохи
        writer.add_scalar('Epoch/Train_Loss', train_loss, epoch)
        writer.add_scalar('Epoch/Train_Accuracy', train_acc, epoch)
        writer.add_scalar('Epoch/Val_Loss', val_loss, epoch)
        writer.add_scalar('Epoch/Val_Accuracy', val_acc, epoch)
        writer.add_scalar('Epoch/LR', optimizer.param_groups[0]['lr'], epoch)

        # Логирование весов и градиентов (каждые 5 эпох)
        if epoch % 5 == 0:
            log_model_weights_and_gradients(model, writer, epoch)

        print(f"\nEpoch {epoch}/{epochs}:")
        print(f"  Train - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%")
        print(f"  Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")
        print(f"  LR: {optimizer.param_groups[0]['lr']:.6f}")

        # Сохранение лучшей модели
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            checkpoint_path = f'checkpoints/{model_name}_best.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
                'val_loss': val_loss,
            }, checkpoint_path)
            print(f"  ✓ Сохранена лучшая модель (val_acc: {val_acc:.2f}%)")

        print("-" * 60)

    # Сохранение финальной модели
    final_checkpoint_path = f'checkpoints/{model_name}_final.pth'
    torch.save({
        'epoch': epochs,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'val_acc': val_acc,
        'val_loss': val_loss,
    }, final_checkpoint_path)

    print("\n" + "="*60)
    print("Обучение завершено!")
    print(f"Лучшая val accuracy: {best_val_acc:.2f}%")
    print(f"Финальная val accuracy: {val_acc:.2f}%")
    print(f"Модели сохранены в директории 'checkpoints/'")
    print(f"Логи TensorBoard: runs/{run_name}")
    print("="*60)

    writer.close()

In [7]:
model = 'cnn'
lr = 0.001

# Запуск обучения
train_model(model, DIR_PATH, 20, 32, lr, 42, 4, False, 5)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Используется устройство: cuda

Режим: ОБЫЧНОЕ ОБУЧЕНИЕ
Загружен датасет:
  - Классы: ['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']
  - Количество классов: 10
  - Обучающая выборка: 4000 изображений
  - Валидационная выборка: 1000 изображений

Создание модели: cnn
Количество обучаемых параметров: 6,799,882

Начало обучения на 20 эпох


Epoch 1 [Val]: 100%|██████████| 32/32 [00:02<00:00, 11.54it/s, loss=1.8902, acc=34.20%]



Epoch 1/20:
  Train - Loss: 2.0242, Acc: 26.05%
  Val   - Loss: 1.7721, Acc: 34.20%
  LR: 0.000994
  ✓ Сохранена лучшая модель (val_acc: 34.20%)
------------------------------------------------------------


Epoch 2 [Val]: 100%|██████████| 32/32 [00:03<00:00, 10.00it/s, loss=1.6575, acc=36.80%]


...


Epoch 18/20:
  Train - Loss: 1.0490, Acc: 61.02%
  Val   - Loss: 1.0956, Acc: 60.00%
  LR: 0.000024
------------------------------------------------------------


Epoch 19 [Val]: 100%|██████████| 32/32 [00:02<00:00, 11.17it/s, loss=1.1555, acc=61.80%]



Epoch 19/20:
  Train - Loss: 1.0434, Acc: 61.05%
  Val   - Loss: 1.0833, Acc: 61.80%
  LR: 0.000006
  ✓ Сохранена лучшая модель (val_acc: 61.80%)
------------------------------------------------------------


Epoch 20 [Val]: 100%|██████████| 32/32 [00:02<00:00, 11.51it/s, loss=1.1575, acc=61.60%]



Epoch 20/20:
  Train - Loss: 1.0513, Acc: 60.92%
  Val   - Loss: 1.0851, Acc: 61.60%
  LR: 0.000000
------------------------------------------------------------

Обучение завершено!
Лучшая val accuracy: 61.80%
Финальная val accuracy: 61.60%
Модели сохранены в директории 'checkpoints/'
Логи TensorBoard: runs/cnn_20251229_041244


In [8]:
model = 'vit'
lr = 0.01

# Запуск обучения
train_model(model, DIR_PATH, 20, 32, lr, 42, 4, False, 5)

Используется устройство: cuda

Режим: ОБЫЧНОЕ ОБУЧЕНИЕ
Загружен датасет:
  - Классы: ['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']
  - Количество классов: 10
  - Обучающая выборка: 4000 изображений
  - Валидационная выборка: 1000 изображений

Создание модели: vit


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

ViT-Tiny модель загружена:
  - Модель: vit_tiny_patch16_224
  - Предобучена: True
  - Размерность признаков: 192
  - Количество классов: 10
  - Backbone заморожен: все параметры requires_grad=False
Количество обучаемых параметров: 1,930
Статистика параметров:
  - Всего: 5,526,346
  - Обучаемых: 1,930
  - Замороженных: 5,524,416

Начало обучения на 20 эпох


Epoch 1 [Val]: 100%|██████████| 32/32 [00:02<00:00, 12.01it/s, loss=0.7483, acc=86.40%]


...


Epoch 18/20:
  Train - Loss: 0.2462, Acc: 93.45%
  Val   - Loss: 0.5250, Acc: 90.10%
  LR: 0.000245
------------------------------------------------------------


Epoch 19 [Val]: 100%|██████████| 32/32 [00:02<00:00, 12.19it/s, loss=0.5396, acc=90.10%]



Epoch 19/20:
  Train - Loss: 0.2259, Acc: 93.90%
  Val   - Loss: 0.5227, Acc: 90.10%
  LR: 0.000062
------------------------------------------------------------


Epoch 20 [Val]: 100%|██████████| 32/32 [00:03<00:00, 10.18it/s, loss=0.5523, acc=90.00%]



Epoch 20/20:
  Train - Loss: 0.2197, Acc: 93.97%
  Val   - Loss: 0.5178, Acc: 90.00%
  LR: 0.000000
------------------------------------------------------------

Обучение завершено!
Лучшая val accuracy: 90.30%
Финальная val accuracy: 90.00%
Модели сохранены в директории 'checkpoints/'
Логи TensorBoard: runs/vit_20251229_042332


In [9]:
"""
Профилировка моделей с использованием torch.profiler.
Сравнение CNN и ViT-Tiny по использованию ресурсов.
"""

import argparse
import os
import torch
import torch.nn as nn
from torch.profiler import profile, record_function, ProfilerActivity
from tqdm import tqdm


def profile_model(
    model: nn.Module,
    data_loader,
    device: torch.device,
    num_steps: int,
    model_name: str,
    output_dir: str = 'results',
    save_tensorboard=False,
    save_chrome_trace=True
):
    """
    Профилирует модель на заданном количестве шагов.

    Args:
        model: Модель для профилировки
        data_loader: DataLoader с данными
        device: Устройство (CPU/GPU)
        num_steps: Количество шагов для профилировки
        model_name: Название модели для сохранения результатов
        output_dir: Директория для сохранения результатов
    """
    model.eval()
    criterion = nn.CrossEntropyLoss()

    os.makedirs(output_dir, exist_ok=True)
    trace_path = os.path.join(output_dir, f'profiler_trace_{model_name}.json')

    print(f"\nПрофилировка модели: {model_name}")
    print(f"Количество шагов: {num_steps}")
    print(f"Устройство: {device}")
    print("-" * 60)

    # Настройка профайлера
    activities = [ProfilerActivity.CPU]
    if torch.cuda.is_available():
        activities.append(ProfilerActivity.CUDA)

    with profile(
        activities=activities,
        record_shapes=True,
        profile_memory=True,
        with_stack=True,
        # Убираем on_trace_ready, чтобы сохранить вручную
    ) as prof:

        data_iter = iter(data_loader)
        pbar = tqdm(range(num_steps), desc='Профилировка')

        for step in pbar:
            try:
                images, labels = next(data_iter)
            except StopIteration:
                data_iter = iter(data_loader)
                images, labels = next(data_iter)

            images, labels = images.to(device), labels.to(device)

            with record_function("forward"):
                with torch.no_grad():
                    outputs = model(images)
                    loss = criterion(outputs, labels)

            prof.step()

    # Сохранение trace в JSON формате для Chrome Tracing
    prof.export_chrome_trace(trace_path)
    print(f"\nTrace сохранен: {trace_path}")
    print(f"Откройте в Chrome: chrome://tracing")

    # Опционально: сохраняем для TensorBoard отдельно
    tensorboard_dir = f'runs/profiler_{model_name}'
    os.makedirs(tensorboard_dir, exist_ok=True)
    print(f"TensorBoard логи: {tensorboard_dir}")

    # Вывод статистики
    print("\n" + "="*60)
    print("СТАТИСТИКА ПО CPU")
    print("="*60)
    print(prof.key_averages().table(
        sort_by="cpu_time_total",
        row_limit=20
    ))

    if torch.cuda.is_available():
        print("\n" + "="*60)
        print("СТАТИСТИКА ПО CUDA")
        print("="*60)
        print(prof.key_averages().table(
            sort_by="cuda_time_total",
            row_limit=20
        ))

    # Сохранение статистики в файл
    stats_path = os.path.join(output_dir, f'profiler_stats_{model_name}.txt')
    with open(stats_path, 'w') as f:
        f.write("="*60 + "\n")
        f.write(f"ПРОФИЛИРОВКА МОДЕЛИ: {model_name}\n")
        f.write("="*60 + "\n\n")

        f.write("СТАТИСТИКА ПО CPU\n")
        f.write("-"*60 + "\n")
        f.write(prof.key_averages().table(
            sort_by="cpu_time_total",
            row_limit=30
        ))
        f.write("\n\n")

        if torch.cuda.is_available():
            f.write("СТАТИСТИКА ПО CUDA\n")
            f.write("-"*60 + "\n")
            f.write(prof.key_averages().table(
                sort_by="cuda_time_total",
                row_limit=30
            ))

    print(f"\nСтатистика сохранена: {stats_path}")

    return prof


def compare_models(
    cnn_prof,
    vit_prof,
    output_dir: str = 'results'
):
    """
    Сравнивает профили двух моделей.

    Args:
        cnn_prof: Профиль CNN модели
        vit_prof: Профиль ViT модели
        output_dir: Директория для сохранения результатов
    """
    print("\n" + "="*60)
    print("СРАВНЕНИЕ МОДЕЛЕЙ")
    print("="*60)

    # Получаем ключевые метрики
    cnn_stats = cnn_prof.key_averages()
    vit_stats = vit_prof.key_averages()

    # Суммарное время CPU
    cnn_cpu_time = sum([item.cpu_time_total for item in cnn_stats])
    vit_cpu_time = sum([item.cpu_time_total for item in vit_stats])

    print(f"\nВремя CPU:")
    print(f"  CNN:      {cnn_cpu_time / 1e6:.2f} ms")
    print(f"  ViT-Tiny: {vit_cpu_time / 1e6:.2f} ms")
    print(f"  Разница:  {abs(cnn_cpu_time - vit_cpu_time) / 1e6:.2f} ms")
    print(f"  Быстрее:  {'CNN' if cnn_cpu_time < vit_cpu_time else 'ViT-Tiny'}")

    if torch.cuda.is_available():
        # Суммарное время CUDA
        cnn_cuda_time = sum([getattr(item, "cuda_time_total", 0) for item in cnn_stats])
        vit_cuda_time = sum([getattr(item, "cuda_time_total", 0) for item in vit_stats])

        print(f"\nВремя CUDA:")
        print(f"  CNN:      {cnn_cuda_time / 1e6:.2f} ms")
        print(f"  ViT-Tiny: {vit_cuda_time / 1e6:.2f} ms")
        print(f"  Разница:  {abs(cnn_cuda_time - vit_cuda_time) / 1e6:.2f} ms")
        print(f"  Быстрее:  {'CNN' if cnn_cuda_time < vit_cuda_time else 'ViT-Tiny'}")

    # Сохранение сравнения
    comparison_path = os.path.join(output_dir, 'profiler_comparison.txt')
    with open(comparison_path, 'w') as f:
        f.write("="*60 + "\n")
        f.write("СРАВНЕНИЕ МОДЕЛЕЙ: CNN vs ViT-Tiny\n")
        f.write("="*60 + "\n\n")

        f.write(f"Время CPU:\n")
        f.write(f"  CNN:      {cnn_cpu_time / 1e6:.2f} ms\n")
        f.write(f"  ViT-Tiny: {vit_cpu_time / 1e6:.2f} ms\n")
        f.write(f"  Разница:  {abs(cnn_cpu_time - vit_cpu_time) / 1e6:.2f} ms\n")
        f.write(f"  Быстрее:  {'CNN' if cnn_cpu_time < vit_cpu_time else 'ViT-Tiny'}\n\n")

        if torch.cuda.is_available():
            f.write(f"Время CUDA:\n")
            f.write(f"  CNN:      {cnn_cuda_time / 1e6:.2f} ms\n")
            f.write(f"  ViT-Tiny: {vit_cuda_time / 1e6:.2f} ms\n")
            f.write(f"  Разница:  {abs(cnn_cuda_time - vit_cuda_time) / 1e6:.2f} ms\n")
            f.write(f"  Быстрее:  {'CNN' if cnn_cuda_time < vit_cuda_time else 'ViT-Tiny'}\n")

    print(f"\nСравнение сохранено: {comparison_path}")

In [10]:
OUTPUT_DIR = Path("./traces")
OUTPUT_DIR.mkdir(exist_ok=True)

In [11]:
model = 'both'
data_dir = DIR_PATH
num_steps = 100
batch_size = 32
seed = 42
output_dir = str(OUTPUT_DIR)


# Установка seed
set_seed(seed)

# Устройство
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используется устройство: {device}")

# Загрузка данных
train_loader, _, num_classes = get_dataloaders(
    data_dir,
    batch_size=batch_size,
    num_workers=2,
    seed=seed
)

profiles = {}

# Профилировка CNN
if model in ['cnn', 'both']:
    print("\n" + "="*60)
    print("ПРОФИЛИРОВКА CNN")
    print("="*60)

    cnn_model = SimpleCNN(num_classes=num_classes).to(device)
    cnn_prof = profile_model(
        cnn_model, train_loader, device,
        num_steps, 'cnn', output_dir
    )
    profiles['cnn'] = cnn_prof

# Профилировка ViT
if model in ['vit', 'both']:
    print("\n" + "="*60)
    print("ПРОФИЛИРОВКА ViT-Tiny")
    print("="*60)

    vit_model = ViTLinearProbe(num_classes=num_classes, pretrained=True).to(device)
    vit_prof = profile_model(
        vit_model, train_loader, device,
        num_steps, 'vit', output_dir
    )
    profiles['vit'] = vit_prof

# Сравнение моделей
if model == 'both':
    compare_models(profiles['cnn'], profiles['vit'], output_dir)

print("\n" + "="*60)
print("ПРОФИЛИРОВКА ЗАВЕРШЕНА")
print("="*60)
print(f"\nРезультаты сохранены в директории: {output_dir}/")
print("\nДля просмотра trace:")
print("  1. Откройте Chrome и перейдите на chrome://tracing")
print(f"  2. Загрузите файл: {output_dir}/profiler_trace_*.json")
print("\nДля просмотра TensorBoard логов:")
print("  tensorboard --logdir runs")

Используется устройство: cuda
Загружен датасет:
  - Классы: ['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']
  - Количество классов: 10
  - Обучающая выборка: 4000 изображений
  - Валидационная выборка: 1000 изображений

ПРОФИЛИРОВКА CNN

Профилировка модели: cnn
Количество шагов: 100
Устройство: cuda
------------------------------------------------------------


Профилировка: 100%|██████████| 100/100 [00:17<00:00,  5.57it/s]



Trace сохранен: traces/profiler_trace_cnn.json
Откройте в Chrome: chrome://tracing
TensorBoard логи: runs/profiler_cnn

СТАТИСТИКА ПО CPU
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
enumerate(DataLoader)#_MultiProcessingDataLoaderIter...        94.51%       16.516s        94.54%

Профилировка: 100%|██████████| 100/100 [00:18<00:00,  5.52it/s]



Trace сохранен: traces/profiler_trace_vit.json
Откройте в Chrome: chrome://tracing
TensorBoard логи: runs/profiler_vit

СТАТИСТИКА ПО CPU
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
enumerate(DataLoader)#_MultiProcessingDataLoaderIter...        75.70%       13.459s        75.75%

In [12]:
"""
Оценка и сравнение моделей CNN и ViT-Tiny.
Вычисление метрик (accuracy, F1), построение confusion matrix.
"""

import os
import csv

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from tqdm import tqdm


def load_model(
    model_path: str,
    model_type: str,
    num_classes: int,
    device: torch.device
) -> nn.Module:
    """
    Загружает модель из чекпоинта.

    Args:
        model_path: Путь к чекпоинту
        model_type: Тип модели ('cnn' или 'vit')
        num_classes: Количество классов
        device: Устройство

    Returns:
        Загруженная модель
    """
    # Создаем модель
    if model_type == 'cnn':
        model = SimpleCNN(num_classes=num_classes)
    elif model_type == 'vit':
        model = ViTLinearProbe(num_classes=num_classes, pretrained=False)
    else:
        raise ValueError(f"Неизвестный тип модели: {model_type}")

    # Загружаем веса
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    model.eval()

    print(f"Модель {model_type} загружена из {model_path}")
    if 'val_acc' in checkpoint:
        print(f"  Сохраненная val accuracy: {checkpoint['val_acc']:.2f}%")

    return model


def evaluate_model(
    model: nn.Module,
    data_loader,
    device: torch.device,
    class_names: list
) -> dict:
    """
    Оценивает модель на данных.

    Args:
        model: Модель для оценки
        data_loader: DataLoader с данными
        device: Устройство
        class_names: Список названий классов

    Returns:
        Словарь с метриками и предсказаниями
    """
    model.eval()

    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc='Оценка'):
            images = images.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            _, preds = outputs.max(1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probs.extend(probs.cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    # Вычисляем метрики
    accuracy = 100. * (all_preds == all_labels).sum() / len(all_labels)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')

    # Classification report
    report = classification_report(
        all_labels, all_preds,
        target_names=class_names,
        digits=4
    )

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'predictions': all_preds,
        'labels': all_labels,
        'probabilities': all_probs,
        'confusion_matrix': cm,
        'classification_report': report
    }


def plot_confusion_matrix(
    cm: np.ndarray,
    class_names: list,
    model_name: str,
    output_path: str,
    normalize: bool = True
):
    """
    Строит и сохраняет confusion matrix.

    Args:
        cm: Confusion matrix
        class_names: Список названий классов
        model_name: Название модели
        output_path: Путь для сохранения
        normalize: Нормализовать ли матрицу
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        fmt = '.2f'
        title = f'Normalized Confusion Matrix - {model_name}'
    else:
        fmt = 'd'
        title = f'Confusion Matrix - {model_name}'

    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm,
        annot=True,
        fmt=fmt,
        cmap='Blues',
        xticklabels=class_names,
        yticklabels=class_names,
        cbar_kws={'label': 'Proportion' if normalize else 'Count'}
    )
    plt.title(title, fontsize=14, fontweight='bold')
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"Confusion matrix сохранена: {output_path}")


def save_metrics_to_csv(
    metrics_dict: dict,
    output_path: str
):
    """
    Сохраняет метрики в CSV файл.

    Args:
        metrics_dict: Словарь с метриками для разных моделей
        output_path: Путь для сохранения CSV
    """
    with open(output_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Model', 'Accuracy (%)', 'Macro F1', 'Parameters'])

        for model_name, metrics in metrics_dict.items():
            writer.writerow([
                model_name,
                f"{metrics['accuracy']:.2f}",
                f"{metrics['macro_f1']:.4f}",
                metrics.get('num_params', 'N/A')
            ])

    print(f"Метрики сохранены: {output_path}")


def compare_models(
    cnn_results: dict,
    vit_results: dict,
    output_dir: str
):
    """
    Сравнивает результаты двух моделей.

    Args:
        cnn_results: Результаты CNN
        vit_results: Результаты ViT
        output_dir: Директория для сохранения
    """
    print("\n" + "="*60)
    print("СРАВНЕНИЕ МОДЕЛЕЙ")
    print("="*60)

    print(f"\nCNN:")
    print(f"  Accuracy: {cnn_results['accuracy']:.2f}%")
    print(f"  Macro F1: {cnn_results['macro_f1']:.4f}")

    print(f"\nViT-Tiny:")
    print(f"  Accuracy: {vit_results['accuracy']:.2f}%")
    print(f"  Macro F1: {vit_results['macro_f1']:.4f}")

    print(f"\nРазница:")
    acc_diff = vit_results['accuracy'] - cnn_results['accuracy']
    f1_diff = vit_results['macro_f1'] - cnn_results['macro_f1']
    print(f"  Accuracy: {acc_diff:+.2f}% ({'ViT лучше' if acc_diff > 0 else 'CNN лучше'})")
    print(f"  Macro F1: {f1_diff:+.4f} ({'ViT лучше' if f1_diff > 0 else 'CNN лучше'})")

    # Сохранение сравнения
    comparison_path = os.path.join(output_dir, 'model_comparison.txt')
    with open(comparison_path, 'w') as f:
        f.write("="*60 + "\n")
        f.write("СРАВНЕНИЕ МОДЕЛЕЙ: CNN vs ViT-Tiny\n")
        f.write("="*60 + "\n\n")

        f.write("CNN:\n")
        f.write(f"  Accuracy: {cnn_results['accuracy']:.2f}%\n")
        f.write(f"  Macro F1: {cnn_results['macro_f1']:.4f}\n\n")

        f.write("ViT-Tiny:\n")
        f.write(f"  Accuracy: {vit_results['accuracy']:.2f}%\n")
        f.write(f"  Macro F1: {vit_results['macro_f1']:.4f}\n\n")

        f.write("Разница:\n")
        f.write(f"  Accuracy: {acc_diff:+.2f}% ({'ViT лучше' if acc_diff > 0 else 'CNN лучше'})\n")
        f.write(f"  Macro F1: {f1_diff:+.4f} ({'ViT лучше' if f1_diff > 0 else 'CNN лучше'})\n\n")

        f.write("="*60 + "\n")
        f.write("CLASSIFICATION REPORT - CNN\n")
        f.write("="*60 + "\n")
        f.write(cnn_results['classification_report'])
        f.write("\n\n")

        f.write("="*60 + "\n")
        f.write("CLASSIFICATION REPORT - ViT-Tiny\n")
        f.write("="*60 + "\n")
        f.write(vit_results['classification_report'])

    print(f"\nСравнение сохранено: {comparison_path}")


In [13]:
model_cnn = './checkpoints/cnn_best.pth'
model_vit = './checkpoints/vit_best.pth'
data_dir = DIR_PATH
batch_size = 32
seed = 42
output_dir = OUTPUT_DIR

# Установка seed
set_seed(seed)

# Устройство
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используется устройство: {device}")

# Создание директории для результатов
os.makedirs(output_dir, exist_ok=True)

# Загрузка данных
_, val_loader, num_classes = get_dataloaders(
    data_dir,
    batch_size=batch_size,
    num_workers=2,
    seed=seed
)

# Получаем названия классов
from torchvision import datasets
val_dataset = datasets.ImageFolder(root=f"{data_dir}/val")
class_names = val_dataset.classes

print(f"\nКлассы: {class_names}")
print(f"Количество классов: {num_classes}")
print(f"Размер валидационной выборки: {len(val_dataset)}")

# Оценка CNN
print("\n" + "="*60)
print("ОЦЕНКА CNN")
print("="*60)

cnn_model = load_model(model_cnn, 'cnn', num_classes, device)
cnn_results = evaluate_model(cnn_model, val_loader, device, class_names)

print(f"\nРезультаты CNN:")
print(f"  Accuracy: {cnn_results['accuracy']:.2f}%")
print(f"  Macro F1: {cnn_results['macro_f1']:.4f}")

# Сохранение confusion matrix для CNN
plot_confusion_matrix(
    cnn_results['confusion_matrix'],
    class_names,
    'CNN',
    os.path.join(output_dir, 'confusion_matrix_cnn.png')
)

# Оценка ViT
print("\n" + "="*60)
print("ОЦЕНКА ViT-Tiny")
print("="*60)

vit_model = load_model(model_vit, 'vit', num_classes, device)
vit_results = evaluate_model(vit_model, val_loader, device, class_names)

print(f"\nРезультаты ViT-Tiny:")
print(f"  Accuracy: {vit_results['accuracy']:.2f}%")
print(f"  Macro F1: {vit_results['macro_f1']:.4f}")

# Сохранение confusion matrix для ViT
plot_confusion_matrix(
    vit_results['confusion_matrix'],
    class_names,
    'ViT-Tiny',
    os.path.join(output_dir, 'confusion_matrix_vit.png')
)

# Сравнение моделей
compare_models(cnn_results, vit_results, output_dir)

# Сохранение метрик в CSV
metrics_dict = {
    'CNN': {
        'accuracy': cnn_results['accuracy'],
        'macro_f1': cnn_results['macro_f1'],
        'num_params': cnn_model.get_num_parameters() if hasattr(cnn_model, 'get_num_parameters') else 'N/A'
    },
    'ViT-Tiny': {
        'accuracy': vit_results['accuracy'],
        'macro_f1': vit_results['macro_f1'],
        'num_params': vit_model.get_num_parameters() if hasattr(vit_model, 'get_num_parameters') else 'N/A'
    }
}

save_metrics_to_csv(
    metrics_dict,
    os.path.join(output_dir, 'metrics.csv')
)

print("\n" + "="*60)
print("ОЦЕНКА ЗАВЕРШЕНА")
print("="*60)
print(f"\nВсе результаты сохранены в директории: {output_dir}/")

Используется устройство: cuda
Загружен датасет:
  - Классы: ['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']
  - Количество классов: 10
  - Обучающая выборка: 4000 изображений
  - Валидационная выборка: 1000 изображений

Классы: ['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']
Количество классов: 10
Размер валидационной выборки: 1000

ОЦЕНКА CNN
Модель cnn загружена из ./checkpoints/cnn_best.pth
  Сохраненная val accuracy: 61.80%


Оценка: 100%|██████████| 32/32 [00:02<00:00, 11.37it/s]



Результаты CNN:
  Accuracy: 61.80%
  Macro F1: 0.6144
Confusion matrix сохранена: traces/confusion_matrix_cnn.png

ОЦЕНКА ViT-Tiny
ViT-Tiny модель загружена:
  - Модель: vit_tiny_patch16_224
  - Предобучена: False
  - Размерность признаков: 192
  - Количество классов: 10
  - Backbone заморожен: все параметры requires_grad=False
Модель vit загружена из ./checkpoints/vit_best.pth
  Сохраненная val accuracy: 90.30%


Оценка: 100%|██████████| 32/32 [00:02<00:00, 12.25it/s]



Результаты ViT-Tiny:
  Accuracy: 90.30%
  Macro F1: 0.9034
Confusion matrix сохранена: traces/confusion_matrix_vit.png

СРАВНЕНИЕ МОДЕЛЕЙ

CNN:
  Accuracy: 61.80%
  Macro F1: 0.6144

ViT-Tiny:
  Accuracy: 90.30%
  Macro F1: 0.9034

Разница:
  Accuracy: +28.50% (ViT лучше)
  Macro F1: +0.2891 (ViT лучше)

Сравнение сохранено: traces/model_comparison.txt
Метрики сохранены: traces/metrics.csv

ОЦЕНКА ЗАВЕРШЕНА

Все результаты сохранены в директории: traces/


In [14]:
!zip -r checkpoints.zip ./checkpoints

  adding: checkpoints/ (stored 0%)
  adding: checkpoints/cnn_best.pth (deflated 8%)
  adding: checkpoints/cnn_final.pth (deflated 8%)
  adding: checkpoints/vit_final.pth (deflated 7%)
  adding: checkpoints/vit_best.pth (deflated 7%)


In [16]:
!zip -r data.zip ./data

Выходные данные были обрезаны до нескольких последних строк (5000).
updating: data/train/cat/000347.png (deflated 0%)
updating: data/train/cat/000896.png (deflated 0%)
updating: data/train/cat/002615.png (stored 0%)
updating: data/train/cat/001063.png (deflated 0%)
updating: data/train/cat/003188.png (stored 0%)
updating: data/train/cat/000961.png (stored 0%)
updating: data/train/cat/001510.png (stored 0%)
updating: data/train/cat/004349.png (stored 0%)
updating: data/train/cat/002726.png (stored 0%)
updating: data/train/cat/004631.png (deflated 0%)
updating: data/train/cat/004101.png (deflated 1%)
updating: data/train/cat/003688.png (stored 0%)
updating: data/train/cat/001919.png (deflated 0%)
updating: data/train/cat/003721.png (deflated 0%)
updating: data/train/cat/002273.png (stored 0%)
updating: data/train/cat/003599.png (deflated 0%)
updating: data/train/cat/001624.png (deflated 2%)
updating: data/train/cat/003906.png (stored 0%)
updating: data/train/cat/001059.png (deflated 0%)


In [22]:
!zip -r runs.zip ./runs

  adding: runs/ (stored 0%)
  adding: runs/profiler_vit/ (stored 0%)
  adding: runs/vit_20251229_042332/ (stored 0%)
  adding: runs/vit_20251229_042332/events.out.tfevents.1766982212.9df43ca8e2d0.365.1 (deflated 88%)
  adding: runs/cnn_20251229_041244/ (stored 0%)
  adding: runs/cnn_20251229_041244/events.out.tfevents.1766981564.9df43ca8e2d0.365.0 (deflated 93%)
  adding: runs/profiler_cnn/ (stored 0%)


In [23]:
!zip -r traces.zip ./traces

  adding: traces/ (stored 0%)
  adding: traces/profiler_stats_vit.txt (deflated 87%)
  adding: traces/model_comparison.txt (deflated 72%)
  adding: traces/profiler_trace_vit.json (deflated 92%)
  adding: traces/profiler_comparison.txt (deflated 58%)
  adding: traces/confusion_matrix_vit.png (deflated 15%)
  adding: traces/metrics.csv (stored 0%)
  adding: traces/profiler_trace_cnn.json (deflated 91%)
  adding: traces/profiler_stats_cnn.txt (deflated 88%)
  adding: traces/confusion_matrix_cnn.png (deflated 13%)
